## Homework 1: Part 2
## Implementing regularized linear regression 
In this part, you will implement regularized linear regression and use it to
study models with different bias-variance properties. 

This notebook contains code that helps you get started on 
linear regression with regularization. 
You will need to complete functions in
**reg_linear_regressor.py** and **utils.py**.
Modify this notebook in places marked by **TODO:**.

## Regularized linear regression: an example
In this problem, you will implement regularized linear regression
to predict the amount of water flowing out of a dam using the change
of water level in a reservoir. We will begin by visualizing the dataset containing historical records on the
change in the water level $x$, and the amount of water $y$, flowing out of the dam.
This dataset is divided into three parts:
- A training set that you will use to learn the model:  X,  y.
- A validation set for determining the regularization parameter: Xval, yval.
- A test set for evaluating the performance of your model:  Xtest, ytest. These are unseen examples that were not used during the training of the model.

Run the cell below and it  will plot the training data as shown in Figure 6 in your assignment handout. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import utils
import plot_utils
from reg_linear_regressor_multi import RegularizedLinearReg_SquaredLoss

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython

%load_ext autoreload
%autoreload 2

########################################################################
## =========== Part 1: Loading and Visualizing Data ===================#
########################################################################
# Load Training Data

print('Loading and Visualizing Data ...')

X, y, Xtest, ytest, Xval, yval = utils.load_mat('ex2data1.mat')

# Plot training data

plot_utils.plot_data(X,y,'Change in water level (x)','Water flowing out of the dam (y)')
plt.show()


## Regularized Linear Regression cost function and gradient (vectorized)
You should now implement the loss function and gradient of the
loss function for regularized linear regression in **reg_linear_regression_multi.py**. Then run the cell below.
Regularized linear regression has the following cost function:
$$J(\theta) = \frac{1}{2m} \left( \sum_{i=1}^{m} {( y^{(i)} - h_\theta(x^{(i)})}^2\right) + \frac{\lambda}{2m}\left( \sum_{j=1}^{n} {\theta_j}^2 \right) $$
where $\lambda$ is a regularization parameter which controls the degree of regularization
(thus, help preventing overfitting). The regularization term puts
a penalty on the overall cost $J(\theta)$. As the magnitudes of the model parameters $\theta_j$ increase, the penalty increases as well. Note that you should not regularize the $\theta_0$ term. You should now complete the code for the method loss in the class Reg_LinearRegression_SquaredLoss in the file reg_linear_regressor_multi.py to calculate $J(\theta)$. Vectorize your code and avoid writing for loops.

Correspondingly, the partial derivative of the regularized linear regression  cost function
with respect to  $\theta_j$ is defined as:
\begin{eqnarray*}
\frac{\partial J(\theta)}{\partial \theta_0} & = & \frac{1}{m} \sum_{i=1}^{m} (h_\theta(x^{(i)}) - y^{(i)}){x_j}^{(i)} \\
\frac{\partial J(\theta)}{\partial \theta_j} & = & \left(\frac{1}{m} \sum_{i=1}^{m} (h_\theta(x^{(i)}) - y^{(i)} ){x_j}^{(i)}\right) + \frac{\lambda}{m}{\theta_j} \; \; \; \mbox{ for }j \geq 1\\
\end{eqnarray*}
You should now complete the code for the method grad_loss in the class {Reg_LinearRegression_SquaredLoss in the file reg_linear_regressor_multi.py}
to  calculate the gradient, returning
it in the variable grad. 

Then evaluate the cell below. It runs the train method in  reg_linear_regressor_multi.py to compute the optimal value
of $\theta$. This training function uses scipy's fmin_bfgs to optimize the cost function.
Here we have set the regularization parameter $\lambda$ to zero. 

The best fit line plotted by the script  tells us that the model is
not a good fit to the data because the data is non-linear. While
visualizing the best fit as shown is one possible way to debug your learning
algorithm, it is not always easy to visualize the data and model. In the next
cell, you will implement a function to generate learning curves that can
help you debug your learning algorithm even if it is not easy to visualize the
data.

In [ ]:
# append a column of ones to matrix X

XX = np.vstack([np.ones((X.shape[0],)),X]).T

#  Train linear regression with lambda = 0

reglinear_reg1 = RegularizedLinearReg_SquaredLoss()
theta_opt0 = reglinear_reg1.train(XX,y,reg=0.0,num_iters=1000)
print('Theta at lambda = 0 is %s' % (theta_opt0))

# plot fit over data and show it (or save it in fig7.pdf)
plot_utils.plot_data(X,y,'Change in water level (x)','Water flowing out of the dam (y)')
plt.plot(X,np.dot(XX,theta_opt0),'g-',linewidth=3)
plt.show()


## Learning curve for linear regression
An important concept in machine learning is the bias-variance tradeoff. Models
with high bias are not complex enough for the data and tend to underfit,
while models with high variance overfit  the training data.
Here you will plot training and test errors on a
learning curve to diagnose bias-variance problems.

A  learning curve plots
training and cross validation error as a function of training set size. You will complete the
function  learning_curve in utils.py so that it returns a vector of errors for the
training set and  validation set.
To obtain different training set sizes,
use different subsets of the original training set X. Specifically, for
a training set size of $i$, you should use the first $i$ examples.

You can use the train function to find the parameter $\theta$. Note
that the regularization $\lambda$  is passed as a parameter to the learning_curve function.
After learning the $\theta$ parameter, you should compute the error on the training
and validation sets. Recall that the training error for a dataset is
defined as:
$$ J(\theta) = \frac{1}{2m} \left( \sum_{i=1}^{m} {( y^{(i)} - h_\theta(x^{(i)})}^2\right)  $$


In particular, note that the training error does not include the regularization
term. One way to compute the training error is to use your existing
cost function and set the regularization parameter reg to 0 only when using it to compute the training error
and validation error. When you are computing the training set error,
make sure you compute it on the training subset
instead of the entire training set. However, for the validation error,
you should compute it over the entire validation set. You should store
the computed errors in the vectors error_train and error_val.
When you are finished, the cell below will print the learning curves and produce
a plot similar to Figure 8 in the assignment handout.

In [ ]:
reg = 1.0
XXval = np.vstack([np.ones((Xval.shape[0],)),Xval]).T

# implement the learning_curve function in utils.py
# this script will run your function and show the learning curve

error_train, error_val = utils.learning_curve(XX,y,XXval,yval,reg)
plot_utils.plot_learning_curve(error_train, error_val,reg)
plt.show()


## Polynomial regression: expanding the basis functions
The problem with our linear model was that it was too simple for the data
and resulted in underfitting (high bias). In this problem, you will address this issue by adding more features. In particular, you will consider hypotheses of the form
$$ h_\theta(x) = \theta_0 + \theta_1 x + \theta_2 x^2 + \ldots + \theta_p x^p $$
This is still a linear model from the point of view of the parameter space. We have augmented the features with powers of $x$.
Code in the cell below builds these features using sklearn's preprocessing module. 

We use a polynomial of degree 6.
It turns out that if we run the training directly on the projected data,  it will
not work well as the features would be badly scaled (e.g., an example with
$x = 40$ will now have a feature $x^6 = 40^6 = 4.1 \times 10^{9}$). Therefore, you will
need to use feature normalization.
Before learning the parameter $\theta$ for the polynomial regression, the script in the cell below will
first call the **feature_normalize** function you wrote earlier. It will normalize the features of the training set,
storing the mu, sigma parameters separately. It will project the validation and test sets too, but normalize them using the same mu and sigma parameters.

In [ ]:
from utils import feature_normalize
import sklearn
from sklearn.preprocessing import PolynomialFeatures

# Map X onto polynomial features and normalize
# We will consider a 6th order polynomial fit for the data

p = 6
poly = sklearn.preprocessing.PolynomialFeatures(degree=p,include_bias=False)
X_poly = poly.fit_transform(np.reshape(X,(len(X),1)))
X_poly, mu, sigma = utils.feature_normalize(X_poly)

# add a column of ones to X_poly

XX_poly = np.vstack([np.ones((X_poly.shape[0],)),X_poly.T]).T

# map Xtest and Xval into the same polynomial features

X_poly_test = poly.fit_transform(np.reshape(Xtest,(len(Xtest),1)))
X_poly_val = poly.fit_transform(np.reshape(Xval,(len(Xval),1)))

# normalize these two sets with the same mu and sigma

X_poly_test = (X_poly_test - mu) / sigma
X_poly_val = (X_poly_val - mu) / sigma

# add a column of ones to both X_poly_test and X_poly_val
XX_poly_test = np.vstack([np.ones((X_poly_test.shape[0],)),X_poly_test.T]).T
XX_poly_val = np.vstack([np.ones((X_poly_val.shape[0],)),X_poly_val.T]).T

## Learning curves for polynomial regression
The cell below trains a linear model on the transformed data. After learning $\theta$, you should see two plots 
generated for polynomial regression with $\lambda= 0$.

In [ ]:
reg = 0.0
reglinear_reg2 = RegularizedLinearReg_SquaredLoss()
theta_opt1 = reglinear_reg1.train(XX_poly,y,reg=reg,num_iters=10000)
print('Theta at lambda = 0 is %s' % (theta_opt1))


# plot data and training fit for the 6th order polynomial 

plot_utils.plot_fit(X,y,np.min(X),np.max(X),mu,sigma,theta_opt1,p,'Change in water level (x)','Water flowing out of dam (y)','Polynomial Regression fit with lambda = 0 and polynomial features of degree = ' + str(p))
plt.show()

# plot learning curve for data (6th order polynomail basis function) 

error_train,error_val = utils.learning_curve(XX_poly,y,XX_poly_val,yval,reg)
plot_utils.plot_learning_curve(error_train,error_val,reg)
plt.show()

## Adjusting the regularization parameter
You will now explore how the regularization parameter
affects the bias-variance of regularized polynomial regression. You should
now modify the  lambda parameter in the cell above and try $\lambda = 1, 10, 100$. For
each of these values, the script will generate a polynomial fit to the data
and also a learning curve. Submit two plots for each value of lambda: the fit as well as the learning curve. Comment on the impact of the choice of lambda on the quality of the learned model in your **writeup.pdf**. 

## Calculating test error on the best model
To get a better indication of a model's performance in the real
world, it is important to evaluate the final model on a test set that was
not used in any part of training (that is, it was neither used to select the regularization parameter, nor to learn the model parameters).
Calculate the error of the best model that you found with the previous analysis
and report it.

In [ ]:
# TODO: your code for computing test error on the best model (model with the best lambda)


## Selecting $\lambda$ using a validation set
You will implement an automated method to select the $\lambda$ parameter. Concretely, you will use a validation set to evaluate how good each $\lambda$ value is. After selecting the best $\lambda$ value using the 
validation set, we can then evaluate the model on the test set to estimate
how well the model will perform on actual unseen data.
**Complete the function validation_curve.m in utils.py**. Specifically,
you should should use the train method on an instance of the class Reg_Linear_Regressor to train the model using
different values of $\lambda$ and to compute the training error and  validation error.
You should try $\lambda$ in the following range: \{0, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3, 10\}.

After you have completed the code, run the cell below to plot a validation curve of $\lambda$ versus the error. This plot  allows you select
which $\lambda$ value to use. Due to randomness
in the training and validation splits of the dataset, the cross validation error
can sometimes be lower than the training error. Submit a pdf version of this plot in **writeup.pdf**. Comment on the best choice of $\lambda$ for this problem.

In [ ]:
# now implement the validation_curve function in utils.py
# this function helps in determining the best lambda using a
# a validation set
# The code will now run your function and plot the figure 

reg_vec, error_train, error_val = utils.validation_curve(XX_poly,y,XX_poly_val,yval)
plot_utils.plot_lambda_selection(reg_vec,error_train,error_val)
plt.show()


## Plotting learning curves with randomly selected examples 
In practice, especially for small training sets, when you plot learning curves
to debug your algorithms, it is often helpful to average across multiple sets
of randomly selected examples to determine the training error and 
validation error.
Concretely, to determine the training error and cross validation error for
$i$ examples, you should first randomly select $i$ examples from the training set
and $i$ examples from the  validation set. You will then learn the model parameters using the randomly chosen training set and evaluate the parameters on the randomly chosen training set and validation set. The above
steps should then be repeated multiple times (say 50) and the averaged error
should be used to determine the training error and cross validation error for
$i$ examples.
Implement the above
strategy for computing the learning curves. Complete the  function
**learning_curve_averaged** in  **utils.py** to generate compute and generate this plot. Then evaluate the cell below.

In [ ]:
# now implement the averaged learning curve function in utils.py
# The script runs your function, plots the curves and saves it in fig11.pdf

reg = 1.0
error_train,error_val = utils.averaged_learning_curve(XX_poly,y,XX_poly_val,yval,reg)
plot_utils.plot_learning_curve(error_train,error_val,reg)
plt.show()


## Comparing the learned parameters of ridge regression and lasso regression
With different regularization methods, the learned parameters behaves differently. In this section you are going to compare the parameters learned from the ridge regression and the lasso regression. Remember that the loss and subgradient of the lasso regression has the form:
$$\begin{eqnarray*}
J(\theta) & = & \frac{1}{2m} \left( \sum_{i=1}^{m} {( y^{(i)} - h_\theta(x^{(i)})}^2\right) + \frac{\lambda}{m} \sum_{j=1}^{n} |{\theta_j}|\\
\frac{\partial J(\theta)}{\partial \theta_0} & = & \frac{1}{m} \sum_{i=1}^{m} (h_\theta(x^{(i)}) - y^{(i)}){x_j}^{(i)} \\
\frac{\partial J(\theta)}{\partial \theta_j} & = & \left(\frac{1}{m} \sum_{i=1}^{m} (h_\theta(x^{(i)}) - y^{(i)} ){x_j}^{(i)}\right) + \frac{\lambda}{m}{\mbox{sign}(\theta_j)} \; \; \; \mbox{ for }j \geq 1\\
\end{eqnarray*}$$First complete the **loss** and **grad_loss** function in the **LassoReg_SquaredLoss** class in **reg_linear_regressor_multi.py**, then train the parameters using **XX_poly**. Use $\lambda = 0.5$. Plot the magnitude of entries of $\theta$ trained from each method in descending order, and report what you see in **writeup.pdf**.

In [ ]:
from reg_linear_regressor_multi import LassoLinearReg_SquaredLoss
reg = 0.5

# Create the two regressors.

ridge_regressor = RegularizedLinearReg_SquaredLoss()
lasso_regressor = LassoLinearReg_SquaredLoss()

# Train them with the polynomial expansion.

ridge_theta = np.abs(ridge_regressor.train(XX_poly,y,reg=reg,num_iters=10000))[1:]
lasso_theta = np.abs(lasso_regressor.train(XX_poly,y,reg=reg,num_iters=10000))[1:]

# Sort the absolute values of the parameters.

sorted_ridge_theta = -np.sort(-ridge_theta)
sorted_lasso_theta = -np.sort(-lasso_theta)

# Plot the parameters.

num_params = len(ridge_theta)
plt.plot(range(num_params), sorted_ridge_theta, label='ridge params')
plt.plot(range(num_params), sorted_lasso_theta, label='lasso params')
plt.legend()
plt.show()

## Extra Credit: Building regularized models for Boston data set (10 points)
-- completely optional

Perform a bias variance analysis of the Boston housing data set with the thirteen predictors, following the steps on the simple data set above. Use sklearn's  built-in functions to split the data into training, validation and test sets. What is the lowest achievable error on the test set with $\lambda = 0$? Select the best value for $\lambda$ and report the test set error with the best $\lambda$. Use the technique of adding features to extend each column of the Boston data set with powers of the values in the column. Repeat the bias-variance analysis with quadratic and cubic features. What is the test set error with quadratic features with the best $\lambda$ chosen with the validation set? What is the test set error with cubic features with the best $\lambda$ chosen with the validation set? Put your analysis code in a separate Python script or notebook called bostonexpt.py or bostonexpt.ipynb. Present your results analytically with plots to support your findings. Discuss the impact of regularization for building good models for the Boston housing data set.

In [ ]:
### TODO: your Boston code here or in a separate notebook.